In [1]:
import spacy

nlp = spacy.load('ja_ginza')
doc = nlp('これはテストです。6月のセミナーです。')

for sent in doc.sents:
    print(sent)
    for token in sent:
        info = [
            token.i,         # トークン番号
            token.orth_,     # テキスト
            token.lemma_,    # 基本形
            token.pos_,      # 品詞
            token.tag_,      # 品詞詳細
        ]
        print(info)

これはテストです。
[0, 'これ', '此れ', 'PRON', '代名詞']
[1, 'は', 'は', 'ADP', '助詞-係助詞']
[2, 'テスト', 'テスト', 'NOUN', '名詞-普通名詞-サ変可能']
[3, 'です', 'です', 'AUX', '助動詞']
[4, '。', '。', 'PUNCT', '補助記号-句点']
6月のセミナーです。
[5, '6', '6', 'NUM', '名詞-数詞']
[6, '月', '月', 'NOUN', '名詞-普通名詞-助数詞可能']
[7, 'の', 'の', 'ADP', '助詞-格助詞']
[8, 'セミナー', 'セミナー', 'NOUN', '名詞-普通名詞-一般']
[9, 'です', 'です', 'AUX', '助動詞']
[10, '。', '。', 'PUNCT', '補助記号-句点']


In [2]:
import spacy

nlp = spacy.load('ja_ginza')

with open('bocchan.txt', 'r',encoding="utf-8", errors="ignore") as f1, open('bocchan2.txt', 'w', encoding="utf-8", errors="ignore") as f2:
    for line in f1:
        for sent in nlp(line).sents:
            for token in sent:
                f2.write(f'{token.i}\t{token.orth_}\t{token.lemma_}\t'
                         f'{token.pos_}\t{token.tag_}\n')
            f2.write('EOS\n')

In [3]:
def sequence_gen():
    ''' 情報を一文ずつ取得 '''
    with open('bocchan2.txt', encoding='utf-8') as f:
        sequence = []
        for line in f:
            if line == 'EOS\n':
                yield sequence
                sequence = []
                continue
            word_info = line.strip().split("\t")
            if len(word_info) ==5:
                pos = word_info[3].split(',')
                pos2 = word_info[4].split(',')
                sequence.append({'surface': word_info[1],
                                 'base': word_info[2],
                                 'pos': pos[0],
                                 'pos2': pos2[0]
                                })

# for seq in sequence_gen():
#     print(seq, "\n")


In [4]:
import re
from collections import Counter


pattern = re.compile('NN+')
l = []

for seq in sequence_gen():
    encode_str = ''.join('N' if w['pos'] in ('NOUN') else '?' for w in seq)
#     print(encode_str)
    for m in pattern.finditer(encode_str):
        l.append(''.join(w['surface'] for w in seq[m.start():m.end()]))

# with open("bocchan_result.txt", 'wt',encoding='utf-8') as f:
#     for w, n in Counter(l).most_common(50):
#         text = str(w) +" "+ str(n) +"件"+ '\n'
#         f.write(text)
for w, n in Counter(l).most_common(50):
    print(str(w) +" "+ str(n) +"件")

赤シャツ 155件
うらなり君 44件
婆さん 29件
坊っちゃん 14件
控所 11件
お婆さん 10件
時間目 9件
奴等 9件
文学士 8件
寄宿生 8件
赤シャツさん 8件
いか銀 7件
うらなり先生 7件
日目 6件
精神的娯楽 6件
お嬢さん 6件
商買 5件
お茶 5件
団子屋 5件
宿直部屋 5件
お嫁 5件
校長室 4件
肝癪 4件
赤手拭 4件
尻持 4件
難有 4件
お目 4件
爺さん 4件
お母さん 4件
日清談判 4件
祝勝会 4件
師範生 4件
親譲り 3件
日前 3件
畳半 3件
物理学校 3件
面倒臭い 3件
馬鹿気 3件
楷子段 3件
かみさん 3件
辛防 3件
お手際 3件
前任者 3件
蒟蒻版 3件
徹頭徹尾 3件
おっ母さん 3件
別嬪さん 3件
校長さん 3件
師範学校 3件
瓦斯燈 3件


In [5]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
text = ' '.join(l)
#日本語のパスらしいです
fpath = "C:/Windows/Fonts/YuGothM.ttc"
wordcloud = WordCloud(background_color="white",#背景を白に
                     font_path=fpath,width = 800,height=600).generate(text)

plt.imshow(wordcloud)

#pngで保存する
wordcloud.to_file("bocchan.png")